In [36]:
import pandas as pd
import glob
import os
import datetime
import altair as alt
alt.data_transformers.enable("vegafusion")

DataTransformerRegistry.enable('vegafusion')

In [37]:
# Bring in baby name data by state

path = '/Users/jiwonkim/Documents/lede/baby-names/popular-baby-names-by-state'
txt_files = glob.glob(os.path.join(path, "*.TXT"))

# Turn txt files into a single dataframe

state_df = pd.DataFrame([], columns=["State", "Sex", "Year", "Name", "Count"])

for file in txt_files: 
    file_df = pd.read_csv(file, sep=",", header=None, 
                 names=["State", "Sex", "Year", "Name", "Count"])
    state_df = pd.concat([state_df, file_df])

In [38]:
# Turn 'Year' and 'Count' columns into integers that we can work with

state_df['Year'] = state_df['Year'].astype(int)
state_df['Year'] = pd.to_datetime(state_df['Year'], format='%Y')
state_df['Year'] = state_df['Year'].dt.year
state_df['Count'] = state_df['Count'].astype(int)
state_df


,State,Sex,Year,Name,Count
0,IN,F,1910,Mary,619
1,IN,F,1910,Helen,324
2,IN,F,1910,Ruth,238
3,IN,F,1910,Dorothy,215
4,IN,F,1910,Mildred,200
...,...,...,...,...,...
34621,DE,M,2023,Tristan,5
34622,DE,M,2023,Victor,5
34623,DE,M,2023,Walker,5
34624,DE,M,2023,Zayden,5


In [39]:
# Bring in national baby name data 

path = '/Users/jiwonkim/Documents/lede/baby-names/popular-baby-names-national'
txt_files = glob.glob(os.path.join(path, "*.txt"))
txt_files

# Turn txt files into a single dataframe

national_df = pd.DataFrame([], columns=["Name", "Sex", "Count", "Year"])

for file in txt_files: 
    file_name = file.split("/")[-1]
    file_df = pd.read_csv(file, sep=",", header=None, 
                 names=["Name", "Sex", "Count"])
    file_df["Year"] = file_name
    national_df = pd.concat([national_df, file_df])

In [40]:
# Turn 'Year' and 'Count' columns into integers that we can work with

national_df["Year"] = national_df['Year'].str.extract('(\d+)').astype(int)
national_df['Year'] = pd.to_datetime(national_df['Year'], format='%Y')
national_df['Year'] = national_df['Year'].dt.year
national_df['Count'] = national_df['Count'].astype(int)
national_df

,Name,Sex,Count,Year
0,Emily,F,25959,2000
1,Hannah,F,23086,2000
2,Madison,F,19968,2000
3,Ashley,F,17998,2000
4,Sarah,F,17713,2000
...,...,...,...,...
32121,Zyheem,M,5,2019
32122,Zykel,M,5,2019
32123,Zyking,M,5,2019
32124,Zyn,M,5,2019


## Unique Names
Playing around with the data for the number of unique names to find trends, especially any that might support the hypothesis that as society becomes more individualistic, baby names are getting more diverse. 

In [41]:
# Number of unique names per each year
unique_names_df = national_df.groupby('Year')['Name'].nunique().reset_index(name='Number of Unique Names')

# Number of baby names registered in that year
babies_born = national_df.groupby('Year')['Count'].sum().reset_index(name='Babies Born')

# Join two dataframes together
unique_names_df = unique_names_df.join(babies_born.set_index('Year'), on='Year')

# Add column that shows ratio of unique names to the number of babies born
unique_names_df['Ratio'] = unique_names_df['Number of Unique Names'] / unique_names_df['Babies Born']

unique_names_df

,Year,Number of Unique Names,Babies Born,Ratio
0,1880,1889,201484,0.009375
1,1881,1829,192688,0.009492
2,1882,2012,221533,0.009082
3,1883,1962,216945,0.009044
4,1884,2158,243461,0.008864
...,...,...,...,...
139,2019,29505,3468791,0.008506
140,2020,28845,3337845,0.008642
141,2021,28967,3384687,0.008558
142,2022,29279,3378611,0.008666


In [42]:
# Number of unique names per each year
unique_names_sex_df = national_df.groupby(['Year', 'Sex'])['Name'].nunique().reset_index(name='Number of Unique Names')

In [43]:
# Chart the trend of number of unique names
alt.Chart(unique_names_df).mark_line().encode(
    x=alt.X('Year:T', axis=alt.Axis(format='%Y')),
    y='Number of Unique Names'
).properties(
    width=600,
    height=400
)


alt.Chart(...)

In [44]:
# Chart the trend of number of unique names, separated by sex

alt.Chart(unique_names_sex_df).mark_line().encode(
    x=alt.X('Year:T', axis=alt.Axis(format='%Y')),
    y='Number of Unique Names', 
    color='Sex'
).properties(
    width=600,
    height=400
)

alt.Chart(...)

In [45]:
alt.Chart(unique_names_df).mark_line().encode(
    x=alt.X('Year:T', axis=alt.Axis(format='%Y')),
    y='Ratio:Q'
).properties(
    width=600,
    height=400
)

alt.Chart(...)

In [57]:
# What is the trend of the number of unique names by state?
unique_names_by_state = state_df.groupby(['Year', 'State'])['Name'].nunique().reset_index()
unique_names_by_state

alt.Chart(unique_names_by_state, title='States with the biggest diversity in baby names').mark_line().encode(
    x=alt.X('Year:T', axis=alt.Axis(format='%Y')),
    y='Name:Q',
    color=alt.Color('State:N', legend=alt.Legend(symbolLimit=10)), 
    tooltip='State'
).properties(
    width=600,
    height=800
).interactive()

alt.Chart(...)

## Common Names
Playing around with the data to take a look at the most common names throughout history (of this dataset) and see if there are any underlying trends.

In [58]:
# Extract top 10 most common male names in the database and track their trends over the years

m_df = national_df[national_df['Sex'] == 'M']
m_df.groupby('Name')['Count'].sum().reset_index(name='Total Count').sort_values('Total Count', ascending=False).head(10)
top_m_names = m_df[m_df['Name'].isin(['James', 'John', 'Robert', 'Michael', 'William', 'David', 'Joseph', 'Richard', 'Charles', 'Thomas'])]
m_names_trend = top_m_names.groupby(['Name', 'Year'])['Count'].sum().reset_index()
m_names_trend['Year'] = pd.to_datetime(m_names_trend['Year'], format='%Y')

alt.Chart(m_names_trend, title='Most Popular Male Names in America').mark_line().encode(
    x=alt.X('Year:T', axis=alt.Axis(format='%Y')),
    y='Count:Q',
    color='Name:N'
).properties(
    width=600,
    height=400
)


alt.Chart(...)

In [60]:
# Extract top 10 most common female names in the database and track their trends over the years

f_df = national_df[national_df['Sex'] == 'F']
grouped = f_df.groupby('Name')['Count'].sum().reset_index(name='Total Count').sort_values('Total Count', ascending=False).head(10)
top_f_names = f_df[f_df['Name'].isin(grouped['Name'])]
f_names_trend = top_f_names.groupby(['Name', 'Year'])['Count'].sum().reset_index()

alt.Chart(f_names_trend, title='Most Popular Female Names in America').mark_line().encode(
    x=alt.X('Year:T', axis=alt.Axis(format='%Y')),
    y='Count:Q',
    color='Name:N'
).properties(
    width=600,
    height=400
)

alt.Chart(...)

In [49]:
# Top 10 most uncommon/unpopular names

# Male: 
m_grouped = m_df.groupby('Name')['Count'].sum().reset_index(name='Total Count').sort_values('Total Count', ascending=False).tail(50)
m_grouped



,Name,Total Count
26363,Liamjohn,5
38162,Suzette,5
26377,Libby,5
38155,Suvam,5
9511,Dayland,5
3035,Arner,5
9509,Dayl,5
26384,Lidio,5
26386,Liedson,5
9506,Dayion,5


In [50]:
# Female:
f_grouped = f_df.groupby('Name')['Count'].sum().reset_index(name='Total Count').sort_values('Total Count', ascending=False).tail(50)
f_grouped

,Name,Total Count
19798,Ellarine,5
19810,Ellawynn,5
63641,Temetria,5
63642,Temetrius,5
34900,Khalli,5
19806,Ellawease,5
19804,Ellasophia,5
19802,Ellasia,5
44948,Megam,5
47973,Narciss,5


## Presidents vs. Kings, According to Baby Naming Trends

In [51]:
# Make list of American presidents and list of British royals, from 1910 (when the data starts) to today.
presidents = ['Joe', 'Biden', 'Donald', 'Trump', 'Barack', 'Obama', 'George', 'Bush', 'Bill', 'Clinton', 'George', 'Bush', 'Ronald', 'Reagan', 'Jimmy', 'Carter', 'Gerald', 'Ford', 'Richard', 'Nixon', 'Lyndon', 'Johnson', 'John', 'Kennedy', 'Dwight', 'Eisenhower', 'Harry', 'Truman', 'Franklin', 'Roosevelt', 'Herbert', 'Hoover', 'Calvin', 'Coolidge', 'Warren', 'Harding', 'Woodrow', 'Wilson', 'William', 'Taft']
royals = ['Charles', 'Elizabeth', 'George', 'Edward', 'William', 'Harry', 'Philip', 'Margaret', 'Anne', 'Andrew', 'Diana', 'Catherine', 'Meghan', 'Charlotte', 'Louis', 'Henry', 'David', 'Victoria', 'Mary', 'Michael', 'Richard', 'Eugenie', 'Beatrice', 'Alexandra', 'Sophia', 'Windsor']

In [52]:
presidents_in_df = national_df[national_df['Name'].isin(presidents)]
royals_in_df = national_df[national_df['Name'].isin(royals)]

In [53]:
named_after_president = president_names.groupby(['Name', 'Year'])['Count'].sum().reset_index(name='Count')

alt.Chart(named_after_president).mark_line().encode(
    x=alt.X('Year:T', axis=alt.Axis(format='%Y')),
    y='Count:Q',
    color='Name:N', 
    tooltip='Name'
).properties(
    width=600,
    height=400,
).interactive()


NameError: name 'president_names' is not defined

In [ ]:
named_after = presidents_in_df.groupby('Year')['Count'].sum().reset_index(name='Same Name as President').set_index('Year')
named_after_royals = royals_in_df.groupby('Year')['Count'].sum().reset_index(name='Same Name as Royal').set_index('Year')
named_after = named_after.join(named_after_royals).reset_index().melt('Year')
named_after.to_csv("babyname_data.csv")


alt.Chart(named_after).mark_line().encode(
    x=alt.X('Year:T', axis=alt.Axis(format='%Y')),
    y=alt.Y('value:Q', title='Number of Babies'),
    color='variable:N', 
)


alt.Chart(...)